# GEOQUERIES

In [1]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import json
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import geopandas as gpd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from bson.json_util import dumps
import numpy as np

In [2]:
#london_loc = {'type': 'Point', 'coordinates':[ -0.0870022262169723, 51.520624794291884]}
#singapore_loc = {'type': 'Point', 'coordinates':[103.84868192263733, 1.2862863534171092]}
#san_francisco_loc = {'type': 'Point', 'coordinates':[ -122.41049370727525,37.79787988216447]}
london_loc1 = [-0.0870022262169723, 51.520624794291884]
singapore_loc1 = [103.84868192263733, 1.2862863534171092]
san_francisco_loc1 =[ -122.41049370727525,37.79787988216447]

In [3]:
metros = 2500

In [4]:
#Crear la conexión con el database de Londres dentro de MongoDB
client = MongoClient("localhost:27017")
db = client.get_database("Ciudades")

In [5]:
ln = db.get_collection("London")
sg = db.get_collection("Singapore")
sf = db.get_collection("San Francisco")


# FUNCIONES

In [6]:
'''
Esta función recibe la coleccion con la que quieres trabajar, la localizacion establecida, el maximo de distancia
que quieres y la classificación que quieres buscar.
Te devuelve sitios cercanos a la ubicación ajustandose a las condiciones(maxima distancia, por classificacion y por
localización)

'''
def coger_sitios_cercanos(collection, location, Classification, max_distance):
    var_schools = {"$and":[{"location2": {"$near": {"$geometry": location,"$minDistance":0,"$maxDistance": max_distance}}},{"Classification": f'{Classification}'}]}
    encontrar_colegios = collection.find(var_schools)
    return list(encontrar_colegios)
    

In [7]:
#Tipos de classificación/Type
#1. School: Type
#2. Technological Startups
#3. Designing Studios
#4. Leisure Places
#5. Travelling Stations
#6. Parks

In [8]:
# A esta función entra la collection de mongoDB con la que queremos trabajar y la localización establecida.
# Esta función nos crea un dataframe nuevo con la distacia de cada sitio a la localización establecida. 
def get_distances(collection, location1):
    query = [{'$geoNear':{'near':location1,
    'distanceField' : "distance",
    'maxDistance': 1,
    'distanceMultiplier': 6371*1000 ,
    'spherical': True}}]
    geoloc = collection.aggregate(query)
    response_json = json.loads(dumps(geoloc))
    df = pd.DataFrame(response_json)
    df1 = df[['Name', 'Classification', 'Type', 'Country', 'Adress', 'Latitude', 'Longitude', 'location2', 'distance']]
    return df1
    


In [9]:
# En estas 3 variables, tenemos el dataframe con la distacia a nuestro punto inicial en metros.
london_distances = get_distances(ln, london_loc1)
san_francisco_distances = get_distances(sf, san_francisco_loc1)
singapore_distances = get_distances(sg, singapore_loc1)

In [10]:
london_distances.head(2)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Finsbury Square,Parks,Park,GB,Finsbury Sq (Finsbury Pavement),51.520694,-0.086512,"{'type': 'Point', 'coordinates': [-0.086511969...",34.776274
1,Finch's,Leisure Places,Pub,GB,12A Finsbury Sq (City Rd),51.521709,-0.087048,"{'type': 'Point', 'coordinates': [-0.087048251...",120.590198


In [11]:
san_francisco_distances.head(2)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Jean Parker Elementary,School,Elementary School,US,840 Broadway,37.797641,-122.411043,"{'type': 'Point', 'coordinates': [-122.411043,...",55.089767
1,Broadway Tunnel East Mini Park,Parks,Park,US,967 Broadway (at Himmelman),37.797354,-122.410502,"{'type': 'Point', 'coordinates': [-122.4105024...",58.444345


In [12]:
singapore_distances.head(2)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Liquid 40,Leisure Places,Pub,SG,Circular Rd,1.286381,103.848842,"{'type': 'Point', 'coordinates': [103.84884158...",20.615074
1,Audition pub,Leisure Places,Pub,SG,76 Circular Road,1.286644,103.848966,"{'type': 'Point', 'coordinates': [103.84896614...",50.772528


In [13]:
london_distances

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Finsbury Square,Parks,Park,GB,Finsbury Sq (Finsbury Pavement),51.520694,-0.086512,"{'type': 'Point', 'coordinates': [-0.086511969...",34.776274
1,Finch's,Leisure Places,Pub,GB,12A Finsbury Sq (City Rd),51.521709,-0.087048,"{'type': 'Point', 'coordinates': [-0.087048251...",120.590198
2,The Flying Horse,Leisure Places,Pub,GB,52 Wilson St (Sun St),51.520375,-0.085176,"{'type': 'Point', 'coordinates': [-0.085176182...",129.349477
3,Singer Tavern,Leisure Places,Bar,GB,1 City Rd,51.522259,-0.086933,"{'type': 'Point', 'coordinates': [-0.086932681...",181.762518
4,Red Lion,Leisure Places,Pub,GB,1 Eldon St (South Pl),51.518843,-0.086181,"{'type': 'Point', 'coordinates': [-0.086181312...",206.137330
...,...,...,...,...,...,...,...,...,...
350,Bright Horizons Mount Carmel Day Nursery and P...,School,Nursery School,GB,18 Eden Grove,51.551394,-0.112786,"{'type': 'Point', 'coordinates': [-0.112786369...",3858.263739
351,Monkey Puzzle Day Nursery,School,Nursery School,GB,"Loxfords House, Highbury Park, N5",51.556389,-0.098601,"{'type': 'Point', 'coordinates': [-0.098600901...",4056.887769
352,City Academy Hackney,School,High School,GB,Homerton Row,51.548704,-0.048845,"{'type': 'Point', 'coordinates': [-0.048845102...",4088.335992
353,Camden School for Girls,School,High School,GB,Sandall Street,51.547697,-0.134395,"{'type': 'Point', 'coordinates': [-0.134395257...",4450.610518


# PONDERACION

In [14]:
# Para escoger la localización, debemos dar importancia a las condiciones escogidas

In [15]:
def eliminar_a_mas_de_1km (df):
    df = df.drop(df[df.distance > 1000].index)
    return df

In [16]:
lnd_final_definitivo = eliminar_a_mas_de_1km(london_distances)

In [17]:
sgp_final_definitivo = eliminar_a_mas_de_1km(singapore_distances)

In [18]:
sf_final_definitivo = eliminar_a_mas_de_1km(san_francisco_distances)

In [19]:
parks_pond = 0.05
school_pond = 0.30
technological_Startups_pond = 0.05
designing_Studios_pond = 0.25
travelling_Stations_pond = 0.2
leisure_pond = 0.15

In [20]:
len(lnd_final_definitivo.Classification.unique())

6

In [21]:
sgp_final_definitivo.Classification.unique()
#Como vemos Singapur no tiene colegios a menos de 1 km, por lo tanto podemos ir descartandolo.

array(['Leisure Places', 'Designing Studios', 'Technological Startups',
       'Parks', 'Travelling Stations'], dtype=object)

In [22]:
len(sf_final_definitivo.Classification.unique())

6

In [28]:
def conditions(df):
    for i in df['Classification']:
        if i == 'Parks':
            return df['distance'] * parks_pond
        elif i == 'Leisure Places':
            return df['distance'] * leisure_pond
        elif i == 'Technological Startups':
            return df['distance'] * technological_Startups_pond
        elif i == 'Designing Studios':
            return df['distance'] * designing_Studios_pond
        elif i == 'Travelling Stations':
            return df['distance'] * travelling_Stations_pond
        elif i == 'School':
            return df['distance'] * school_pond
        
        
        
    

In [29]:
lnd_final_definitivo["valor"] = conditions(lnd_final_definitivo)

In [30]:
lnd_final_definitivo.sample(5)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance,valor
166,The Fox and Anchor,Leisure Places,Pub,GB,115 Charterhouse St (Lindsey St),51.520347,-0.100530,"{'type': 'Point', 'coordinates': [-0.100530048...",936.489505,46.824475
138,Trafik,Leisure Places,Nightclub,GB,331 Old St (Hoxton Sq),51.527047,-0.080405,"{'type': 'Point', 'coordinates': [-0.080405340...",847.494948,42.374747
67,The Whitecross Tap,Leisure Places,Pub,GB,142-146 Whitecross Street,51.524134,-0.093500,"{'type': 'Point', 'coordinates': [-0.0934997, ...",595.253770,29.762689
143,Coq d'Argent,Leisure Places,Pub,GB,No. 1 Poultry,51.513303,-0.091036,"{'type': 'Point', 'coordinates': [-0.091035806...",860.628806,43.031440
70,The Otherist,Leisure Places,Bar,GB,111 Old Broad St,51.515387,-0.084843,"{'type': 'Point', 'coordinates': [-0.084843, 5...",601.273655,30.063683


In [31]:
sf_final_definitivo['valor'] = conditions(sf_final_definitivo)

In [32]:
sf_final_definitivo.sample(5)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance,valor
81,Cathedral School for Boys,School,Elementary School,US,1275 Sacramento St (Jones),37.792128,-122.414404,"{'type': 'Point', 'coordinates': [-122.4144037...",726.058740,217.817622
131,Costa Brown Architecture,Designing Studios,Design Studio,US,1620 Montgomery St (Lombard),37.804157,-122.404694,"{'type': 'Point', 'coordinates': [-122.4046936...",864.239789,259.271937
137,Irish Times,Leisure Places,Irish Pub,US,500 Sacramento St (at Sansome St.),37.794010,-122.401690,"{'type': 'Point', 'coordinates': [-122.40169, ...",885.179498,265.553849
38,Sutra Nightclub & Lounge,Leisure Places,Nightclub,US,443 Broadway (btwn Montgomery & Kearny),37.797950,-122.404346,"{'type': 'Point', 'coordinates': [-122.4043464...",540.175780,162.052734
107,Mashape HQ,Technological Startups,Tech Startup,US,500 Montgomery St,37.793896,-122.402937,"{'type': 'Point', 'coordinates': [-122.4029373...",798.160205,239.448062


In [34]:
# By multiplyng the distance by the average ponderation given, we give certain importance to eache place.
# Now, we som all the values together an see which place is the best location.

# SUMA PONDERACIONES 

In [37]:
san_francisco_value = sf_final_definitivo.valor.sum()

In [38]:
london_value = lnd_final_definitivo.valor.sum()

In [57]:
san_francisco_value

40627.75581183808

In [58]:
london_value


6204.168892842272

In [59]:
# We have decided to set up our business in London.